# SIGNOR Pipeline

This notebook creates a Network Set of SIGNOR pathway networks using the latest data downloaded via the SIGNOR REST service. SIGNOR is updated regularly and so it also demonstrates how to update the networks in the set.

## Imports Python Packages Needed in the Pipeline

In [ ]:
#!pip uninstall ndexutil -y
#!pip install ndexutil --no-cache-dir
#!pip uninstall ndex2 -y
#!pip uninstall ndex2-dev -y
#!pip install ndex2-dev --no-cache-dir
#!pip freeze

In [1]:
import ndex2 # The ndex2 Python client
from ndex2.cx.aspects import ATTRIBUTE_DATA_TYPE
import ndex2.client as nc
import itertools # convenient iteration utilities 
import requests
import json
import pandas as pd
import io
import sys
import jsonschema
import os
import nicecxModel
#from nicecxModel.cx.aspects import ATTRIBUTE_DATA_TYPE
from datetime import datetime
#import datetime
import networkx as nx
sys.path.append('../../resources')
from tutorial_utils import load_tutorial_config


In [2]:
import ndexutil.tsv.tsv2nicecx as t2n

In [3]:
signor_list_url = 'https://signor.uniroma2.it/getPathwayData.php?list'
#response_list = requests.get(signor_list_url)
#signor_mapping_list = response_list.text
cols = ['pathway_id', 'pathway_name']
signor_mapping_list_df = pd.read_csv(signor_list_url, sep="\t", names = cols)
print(signor_mapping_list_df)


                 pathway_id                                  pathway_name
0                 SIGNOR-AC                                  Adipogenesis
1                 SIGNOR-AD                             Alzheimer Disease
2                SIGNOR-AML                        Acute Myeloid Leukemia
3               SIGNOR-AMPK                                AMPK Signaling
4          SIGNOR-Autophagy                                     Autophagy
5                SIGNOR-CRC                          Colorectal Carcinoma
6                 SIGNOR-DR                      Death Receptor Signaling
7                SIGNOR-EGF                                          EGFR
8             SIGNOR-FapBMP                                      FAP: BMP
9             SIGNOR-FapIL4                                      FAP: IL4
10            SIGNOR-FapINS            FAP: insulin-mediated adipogenesis
11              SIGNOR-FSGS                                          FSGS
12        SIGNOR-G1-S_trans           

## NDEx Access

Configure your NDEx connection based on server, account and password. 

See ["Using NDEx Tutorial Config Files.ipynb"] for more information.

In [4]:
# get the connection parameters from the ndex_tutorial_config.json file in your home directory.
# edit the line below to specify a different connection in the config file
my_server, my_username, my_password = load_tutorial_config("main")

# alternatively, edit and uncomment these lines to set the connection parameters manually 
# my_server = "public.ndexbio.org"
# my_username = None
# my_password = None

my_ndex=ndex2.client.Ndex2(my_server, my_username, my_password)

In [5]:
def get_signor_update_mapping(server, username, password):
    signor_mapping_set = {}
    signor_list_url = 'https://signor.uniroma2.it/getPathwayData.php?list'
    cols = ['pathway_id', 'pathway_name']

    signor_mapping_list_df = pd.read_csv(signor_list_url, sep="\t", names=cols)
    signor_dict = signor_mapping_list_df.to_dict()

    id_to_name = {}
    pathway_names = signor_dict.get('pathway_name')
    pathway_id = signor_dict.get('pathway_id')
    for k, v in pathway_names.items():
        signor_mapping_set[v] = k
        id_to_name[pathway_id.get(k)] = v
        
    my_ndex = nc.Ndex2(server, username, password)
    networks = my_ndex.get_network_summaries_for_user(username)
    update_mapping = {}
    for nk in networks:
        if signor_mapping_set.get(nk.get('name')) is not None or nk.get('name').upper() == 'FULL SIGNOR':
            update_mapping[nk.get('name').upper()] = nk.get('externalId')

    return (update_mapping, id_to_name)

update_signor_mapping, signor_id_name_mapping = get_signor_update_mapping(my_server, my_username, my_password)
print(update_signor_mapping)

{'FULL SIGNOR': 'bb8ddc4c-1cd8-11e8-b939-0ac135e8bacf', 'WNT SIGNALING AND MYOGENESIS': '0bc1e96b-1cc6-11e8-b939-0ac135e8bacf', 'WNT SIGNALING': '0a1bda88-1cc6-11e8-b939-0ac135e8bacf', 'TOL LIKE RECEPTORS': '08583185-1cc6-11e8-b939-0ac135e8bacf', 'TGFBETA SIGNALING': '068b39b2-1cc6-11e8-b939-0ac135e8bacf', 'T CELL ACTIVATION': '04dd629f-1cc6-11e8-b939-0ac135e8bacf', 'THYROID CANCER': '030fa77c-1cc6-11e8-b939-0ac135e8bacf', 'TNF ALPHA': '016cf3f9-1cc6-11e8-b939-0ac135e8bacf', 'SATELLITE: ATROPHY': 'ffcba006-1cc5-11e8-b939-0ac135e8bacf', 'SAPK/JNK SIGNALING': 'fe24cdd3-1cc5-11e8-b939-0ac135e8bacf', 'RHABDOMYOSARCOMA': 'fc73e980-1cc5-11e8-b939-0ac135e8bacf', 'PI3K/AKT': 'fa8cd9fd-1cc5-11e8-b939-0ac135e8bacf', 'PARKINSON DISEASE': 'f8e7193a-1cc5-11e8-b939-0ac135e8bacf', 'PROSTATE CANCER': 'f73b3df7-1cc5-11e8-b939-0ac135e8bacf', 'P38 SIGNALING AND MYOGENESIS': 'f56879c4-1cc5-11e8-b939-0ac135e8bacf', 'P38 SIGNALING': 'f395dca1-1cc5-11e8-b939-0ac135e8bacf', 'NOONAN SYNDROME': 'f1b9074e-1cc5-1

## Get the Pathway Ids from the Mapping File:

In [6]:
def get_signor_network_ids():
    #path = "signor-path_mapping_file.txt"
    #return pd.read_csv(path, sep="\t")

    signor_list_url = 'https://signor.uniroma2.it/getPathwayData.php?list'
    #response_list = requests.get(signor_list_url)
    #signor_mapping_list = response_list.text
    cols = ['pathway_id', 'pathway_name']
    return pd.read_csv(signor_list_url, sep="\t", names = cols)

network_id_dataframe = get_signor_network_ids()
# network_id_dataframe

## Get the Load Plan:

(TODO: validate load plan with jsonschema)

In [7]:
try:
    path_to_load_plan = 'signor_load_plan.json'
    load_plan = None
    with open(path_to_load_plan, 'r') as lp:
        load_plan = json.load(lp)
        
except jsonschema.ValidationError as e1:
    print("Failed to parse the loading plan: " + e1.message)
    print('at path: ' + str(e1.absolute_path))
    print("in block: ")
    print(e1.instance)

## Function to Create NiceCX from SIGNOR Pathway Data

In [10]:
# human_tax_id = "9606"

def get_signor_network(pathway_id, load_plan):
    # TODO - add context (normalize?)
    signor_context = [{
        'ncbigene': 'http://identifiers.org/ncbigene/',
        'hgnc.symbol': 'http://identifiers.org/hgnc.symbol/',
        'uniprot': 'http://identifiers.org/uniprot/',
        'cas': 'http://identifiers.org/cas/',
        'chebi': 'http://identifiers.org/chebi/'}]
    # ncx.set_context(context)

    # parameters = human_tax_id + "organism=" + organism_id + "&id=" + pathway_id
    # pathway_data = requests.get("http://signor.uniroma2.it/getData.php?" + parameters)
    url = "http://signor.uniroma2.it/getPathwayData.php?pathway=" + pathway_id + "&relations=only"
    # print(url)
    response = requests.get(url)
    pathway_data = response.text
    
    #header = [h.strip() for h in pathway_data.readline().split('\t')]
    #print(pathway_data)
    # converters={'CUSTOMER': str, 'ORDER NO': str}
    #converters = {}
    usecols = ["entitya", "typea", "ida", "entityb", "typeb", "idb", "effect", "mechanism", "residue", "sequence", "tax_id", "cell_data", "tissue_data", "pmid", "direct", "notes", "annotator", "sentence"]
    #usecols = ["ENTITYA", "TYPEA", "IDA", "ENTITYB", "TYPEB", "IDB", "EFFECT", "MECHANISM", "RESIDUR", 
    #           "SEQUENCE", "TAX_ID", "CELL_DATA", "TISSUE_DATA", "PMID", "DIRECT", "NOTES", "ANNOTATOR", "SETENCE"]
    #for col in usecols:
    #    converters[col] = str
    # dataframe = pd.read_csv(io.StringIO(pathway_data), sep='\t',converters = converters,usecols = usecols)
    dataframe = pd.read_csv(io.StringIO(pathway_data), 
                dtype=str, 
                na_filter=False, 
                delimiter='\t', 
                engine='python')
                #names=usecols)
    
    #print(dataframe)
    #print(dataframe)
    # filter dataframe to remove rows that are not human
    human_dataframe = dataframe #.loc[dataframe["tax_id"] == "9606"]

    #print(human_dataframe)
    # upcase column names
    rename = {}
    for column_name in human_dataframe.columns:
        rename[column_name]= column_name.upper()
    
    human_dataframe = human_dataframe.rename(columns=rename)

    # df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
    #return human_dataframe
    
    network = t2n.convert_pandas_to_nice_cx_with_load_plan(human_dataframe, load_plan)

    #network.set_network_attribute("SIGNOR_ID", values=pathway_id)

    # Fix values for "DIRECT"
    for edge_id, edge in network.get_edges():
        direct = network.get_edge_attribute(edge_id, "DIRECT")
        # print(direct)
        if direct:
            if direct == "t":
                network.set_edge_attribute(edge, "DIRECT", "YES")
            else:
                network.set_edge_attribute(edge, "DIRECT", "NO")
                
    # Set prefixes for represents based on the "DATABASE" attribute
    #
    #   Note that this is a good example of a situation that calls
    #   for custom code and does not justify an extension to the load_plan
    #   Cases of this type are too variable. Custom code is easier.
    #
    for node_id, node in network.get_nodes():
        database = network.get_node_attribute(node_id, "DATABASE")
        represents = node.get_node_represents()
        if database == "UNIPROT":
            represents = "uniprot:" + represents
            node.set_node_represents(represents)
        if database == "SIGNOR":
            represents = "signor:" + represents
            node.set_node_represents(represents)
        # in all other cases, the identifier is already prefixed
        network.remove_node_attribute(node_id, "DATABASE")

    #=================================
    # POST PROCESS EDGE ATTRIBUTES
    # Rename citation_ids to citation
    #=================================
    for edge_id, edge in network.get_edges():
        cit1 = network.get_edge_attribute_objects(edge_id, "citation_ids")
        if cit1 is not None:
            network.add_edge_attribute(property_of=cit1.get_property_of(), name='citation', 
                                       values=cit1.get_values(), type=cit1.get_data_type())
            network.remove_edge_attribute(edge_id, "citation_ids")
        
        cd = network.get_edge_attribute_objects(edge_id, "CELL_DATA")
        if cd is not None:
            cd_value = cd.get_values()
            cd_split = cd_value.split(';')
            cd.set_values(cd_split)

            cd.set_data_type('list_of_string')

            
        td = network.get_edge_attribute_objects(edge_id, "TISSUE_DATA")
        if td is not None:
            td_value = td.get_values()
            td_split = td_value.split(';')
            td.set_values(td_split)

            td.set_data_type('list_of_string')

        
    print(network.get_summary())
    return network

signor_network = get_signor_network("SIGNOR-MM", load_plan)
#signor_network = get_signor_network("SIGNOR-MonoIL10TR", load_plan)
#print(signor_network.__str__())

Name: Untitled
Nodes: 30
Edges: 137
Node Attributes: 30
Edge Attributes: 840



## Function to Get the Description and Other Info About the Pathway

In [11]:
def add_pathway_info(network, network_id):
    url = "http://signor.uniroma2.it/getPathwayData.php?pathway=" + str(network_id)#network.get_network_attribute("SIGNOR_ID"))
    print(url)
    response = requests.get(url)
    pathway_info = response.text
    dataframe = pd.read_csv(io.StringIO(pathway_info), sep='\t')
    print(pd.isnull(dataframe.iat[0,1]))
    #print(dataframe)
    if not pd.isnull(dataframe.iat[0, 1]):
        network.set_name(dataframe.iat[0,1])
    if not pd.isnull(dataframe.iat[0, 0]):
        network.set_network_attribute("labels", [dataframe.iat[0, 0]], type='list_of_string')
    if not pd.isnull(dataframe.iat[0, 3]):
        network.set_network_attribute("author", dataframe.iat[0, 3])
    if not pd.isnull(dataframe.iat[0, 2]):
        append_desc = '<p><br/></p><h6><b>Node Legend:</b><br/>Light green oval &gt; Protein/Protein Family<br/>Dark green round rectangle &gt; Complex<br/>Orange octagon &gt; Chemical<br/>Purple octagon &gt; Small molecule<br/>White rectangles &gt; Phenotype<br/>Light blue diamond &gt; Stimulus</h6><h6><b>Edge Legend:</b><br/>Solid &gt; Direct interaction<br/>Dashed &gt; Indirect or Unknown interaction<br/>Blue &gt; Up-regulation<br/>Red &gt; Down-regulation<br/>Black &gt; Form complex or Unknown</h6>'
        network.set_network_attribute("description", '%s %s' % (dataframe.iat[0, 2], append_desc))
        
    network.set_network_attribute('rightsHolder', 'Prof. Gianni Cesareni')
    network.set_network_attribute("rights", "Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)")
    network.set_network_attribute("reference", "<div>Perfetto L., <i>et al.</i></div><div><b>SIGNOR: a database of causal relationships between biological entities</b><i>.</i></div><div>Nucleic Acids Res. 2016 Jan 4;44(D1):D548-54</div><div><span><a href=\"https://doi.org/10.1093/nar/gkv1048\" target=\"\">doi: 10.1093/nar/gkv1048</a></span></div>")    
    network.set_network_attribute('dataSource', 'https://signor.uniroma2.it/pathway_browser.php?organism=&pathway_list=' + str(network_id))
    
    network.set_network_attribute("version", f"{datetime.now():%d-%b-%Y}")
    
    disease_pathways = ['ALZHAIMER DISEASE', 'FSGS', 'NOONAN SYNDROME', 'PARKINSON DISEASE']
    
    cancer_pathways = ['ACUTE MYELOID LEUKEMIA', 'COLORECTAL CARCINOMA', 'GLIOBLASTOMA MULTIFORME', 
                       'LUMINAL BREAST CANCER', 'MALIGNANT MELANOMA', 'PROSTATE CANCER', 
                       'RHABDOMYOSARCOMA', 'THYROID CANCER']
    
    if signor_id_name_mapping.get(network_id).upper() in disease_pathways:
        network.set_network_attribute("networkType", "Disease Pathway")
    elif signor_id_name_mapping.get(network_id).upper() in cancer_pathways:
        network.set_network_attribute("networkType", "Cancer Pathway")
    else:
        network.set_network_attribute("networkType", "Signalling Pathway")
    # TODO: set “networkType” property depending on network
    #    a. Signalling Pathway
    #    b. Disease Pathway
    #    c. Cancer Pathway
    
#add_pathway_info(signor_network, 'SIGNOR-PC')
add_pathway_info(signor_network, 'SIGNOR-MM')
#print(network.get_network_attribute("description"))
print(signor_network.get_network_attribute("Labels"))


http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-MM
False
None


## Specify the Network Containing the Visual Style to Apply to the SIGNOR Networks 

In [12]:
# Use the visual properties of network ... to style each output network
# http://www.ndexbio.org/#/network/d3c5ca09-bb42-11e7-94d3-0ac135e8bacf
# cytoscape_visual_properties_template_id = 'f54eaef9-013c-11e8-81c8-06832d634f41'
cytoscape_visual_properties_template_id = '13c2e3f1-11cc-11e8-b939-0ac135e8bacf' # PUBLIC
# cytoscape_visual_properties_template_id = 'cded1818-1c0d-11e8-801d-06832d634f41' # DEV



## Function to Apply a Layout to the Network

In [13]:
def cartesian(G):
    return [{'cartesianLayout': [
        {'node': n, 'x': float(G.pos[n][0]), 'y': float(G.pos[n][1])}
        for n in G.pos
        ]}]

def apply_spring_layout(network):
    my_networkx = network.to_networkx()
    my_networkx.pos = nx.drawing.spring_layout(my_networkx)
    #my_networkx.pos = nx.drawing.circular_layout(my_networkx)
    cartesian_aspect = cartesian(my_networkx)
    network.set_opaque_aspect("cartesianCoordinates", cartesian_aspect)

## Function to Print a Sample of Nodes in the Network
#### (For a Spot-Check when Debugging)

In [14]:
def spot_check_nodes(network, attribute_name):
    for id, node in itertools.islice(network.get_nodes(), 5):
        attribute_value = my_network.get_node_attribute(node, attribute_name)
        print("%s: %s = %s" % (node.get_name(), attribute_name, attribute_value))

## Function to Upload the Network

In [15]:
def upload_signor_network(network, server, username, password, update_uuid=False):
    if update_uuid:
        message = network.update_to(update_uuid, server, username, password)      
    else:
        message = network.upload_to(server, username, password)
    return(message)

## Function to Process One Signor Pathway ID

In [16]:
def process_signor_id(signor_id, 
                      cytoscape_visual_properties_template_id, 
                      load_plan, 
                      server, 
                      username, 
                      password):
    network = get_signor_network(signor_id, load_plan)
    #print(network.__str__())
    add_pathway_info(network, signor_id)
    #print(network.to_cx())
    network.apply_template(
        username=username, 
        password=password, 
        server=server, 
        uuid=cytoscape_visual_properties_template_id)
    apply_spring_layout(network)
    network.generate_metadata_aspect()
    
    if network.node_int_id_generator:
        network.node_id_lookup = list(network.node_int_id_generator)
    network.generate_aspect('nodes') 
    network.generate_aspect('edges') 
    print(network.get_name())
    network_update_key = update_signor_mapping.get(network.get_name().upper())
    if network_update_key is not None:
        print("updating")
        return upload_signor_network(network, server, username, password, update_uuid=network_update_key)
    else:
        print("new network")
        return upload_signor_network(network, server, username, password)

process_signor_id("SIGNOR-MM", cytoscape_visual_properties_template_id, load_plan, my_server, my_username, my_password)

Name: Untitled
Nodes: 30
Edges: 137
Node Attributes: 30
Edge Attributes: 840

http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-MM
False
Malignant Melanoma
updating


''

## Create a Network Set to Collect the Output Networks

In [17]:
net_set_url = my_ndex.create_networkset('Signor Networks' + f"{datetime.now():%d-%b-%Y-%s}",#str(datetime.now()), 
                                              'Networks from Signor using data obtained by SIGNOR REST API')
net_set_uuid = net_set_url.split('/')[-1]
print('Network set uuid: ' + net_set_uuid)

Network set uuid: 4b5ddfe2-1d81-11e8-b939-0ac135e8bacf


## Process the SIGNOR Pathway IDs

In [18]:
count = 0
limit = 400
signor_uuids = []
#print(network_id_dataframe)
for pathway_id in network_id_dataframe['pathway_id']:
    print(pathway_id)
    upload_message = process_signor_id(
        pathway_id, 
        cytoscape_visual_properties_template_id, 
        load_plan, 
        my_server, 
        my_username, 
        my_password)
    print(upload_message)
    network_uuid = upload_message.split('/')[-1]
    signor_uuids.append(network_uuid)
    if limit:
        count += 1
        if count >= limit:
            break

#spot_check_nodes(signor_uuids[0], "TYPE")
#print('Adding networks to network set')
#my_ndex.add_networks_to_networkset(net_set_uuid, signor_uuids)        
print('Done')

SIGNOR-AC
Name: Untitled
Nodes: 30
Edges: 128
Node Attributes: 30
Edge Attributes: 848

http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-AC
False
Adipogenesis
updating

SIGNOR-AD
Name: Untitled
Nodes: 33
Edges: 121
Node Attributes: 33
Edge Attributes: 779

http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-AD
False
Alzheimer Disease
updating

SIGNOR-AML
Name: Untitled
Nodes: 44
Edges: 183
Node Attributes: 44
Edge Attributes: 1144

http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-AML
False
Acute Myeloid Leukemia
updating

SIGNOR-AMPK
Name: Untitled
Nodes: 26
Edges: 96
Node Attributes: 26
Edge Attributes: 615

http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-AMPK
False
AMPK Signaling
updating

SIGNOR-Autophagy
Name: Untitled
Nodes: 24
Edges: 34
Node Attributes: 24
Edge Attributes: 179

http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-Autophagy
False
Autophagy
updating

SIGNOR-CRC
Name: Untitled
Nodes: 29
Edges: 129
Node Attributes: 29


In [20]:
def process_full_signor(cytoscape_visual_properties_template_id, 
                      load_plan, 
                      server, 
                      username, 
                      password):
    network = get_full_signor_network(load_plan)
    network.set_name('Full Signor')
    #add_pathway_info(network, signor_id)
    #print(network.to_cx())
    network.apply_template(
        username=username, 
        password=password, 
        server=server, 
        uuid=cytoscape_visual_properties_template_id)
    apply_spring_layout(network)

    network_update_key = update_signor_mapping.get(network.get_name().upper())
    if network_update_key is not None:
        print("updating")
        return upload_signor_network(network, server, username, password, update_uuid=network_update_key)
    else:
        print("new network")
        return upload_signor_network(network, server, username, password)

def get_full_signor_network(load_plan):
    # TODO - add context (normalize?)
    signor_context = [{
        'ncbigene': 'http://identifiers.org/ncbigene/',
        'hgnc.symbol': 'http://identifiers.org/hgnc.symbol/',
        'uniprot': 'http://identifiers.org/uniprot/',
        'cas': 'http://identifiers.org/cas/'}]

    url = "http://signor.uniroma2.it/getData.php?organism=9606"

    #response = requests.get(url)
    #pathway_data = response.text
    
    with open('getDataAll.txt', 'r') as tsvfile:
        header = [h.strip() for h in tsvfile.readline().split('\t')]
        usecols = ['entitya','typea','ida','databasea','entityb','typeb','idb','databaseb','effect',
                   'mechanism','residue','sequence','tax_id','cell_data','tissue_data','modulator_complex',
                   'target_complex','modificationa','modaseq','modificationb','modbseq','pmid',
                   'direct','notes','annotator','sentence','signor_id']
        #usecols = ["entitya", "typea", "ida", "entityb", "typeb", "idb", "effect", "mechanism", "residue", "sequence", "tax_id", "cell_data", "tissue_data", "pmid", "direct", "notes", "annotator", "sentence"]
        df = pd.read_csv(tsvfile, 
                    dtype=str, 
                    na_filter=False, 
                    delimiter='\t', 
                    engine='python',
                     names=usecols, index_col=False)
        #df = pd.read_csv(tsvfile,delimiter='\t',engine='python',names=header)
    
        human_dataframe = df[(df["entitya"] != "") & (df["entityb"] != "") & (df["ida"] != "") & (df["idb"] != "")]
        
        #print(human_dataframe)

    
    #human_dataframe = pd.read_csv(io.StringIO(pathway_data), 
    #            dtype=str, 
    #            na_filter=False, 
    #            delimiter='\t', 
    #            engine='python')
                #names=usecols)
    
    # upcase column names
    rename = {}
    for column_name in human_dataframe.columns:
        rename[column_name]= column_name.upper()
    
    human_dataframe = human_dataframe.rename(columns=rename)

    network = t2n.convert_pandas_to_nice_cx_with_load_plan(human_dataframe, load_plan)

    # Fix values for "DIRECT"
    for edge_id, edge in network.get_edges():
        direct = network.get_edge_attribute(edge_id, "DIRECT")
        # print(direct)
        if direct:
            if direct == "t":
                network.set_edge_attribute(edge, "DIRECT", "YES")
            else:
                network.set_edge_attribute(edge, "DIRECT", "NO")
                
    # Set prefixes for represents based on the "DATABASE" attribute
    #
    #   Note that this is a good example of a situation that calls
    #   for custom code and does not justify an extension to the load_plan
    #   Cases of this type are too variable. Custom code is easier.
    #
    for node_id, node in network.get_nodes():
        database = network.get_node_attribute(node_id, "DATABASE")
        represents = node.get_node_represents()
        if database == "UNIPROT":
            represents = "uniprot:" + represents
            node.set_node_represents(represents)
        if database == "SIGNOR":
            represents = "signor:" + represents
            node.set_node_represents(represents)
        # in all other cases, the identifier is already prefixed
        network.remove_node_attribute(node_id, "DATABASE")

    #=================================
    # POST PROCESS EDGE ATTRIBUTES
    # Rename citation_ids to citation
    #=================================
    for edge_id, edge in network.get_edges():
        cit1 = network.get_edge_attribute_objects(edge_id, "citation_ids")
        if cit1 is not None:
            network.add_edge_attribute(property_of=cit1.get_property_of(), name='citation', 
                                       values=cit1.get_values(), type=cit1.get_data_type())
            network.remove_edge_attribute(edge_id, "citation_ids")
        
        cd = network.get_edge_attribute_objects(edge_id, "CELL_DATA")
        if cd is not None:
            cd_value = cd.get_values()
            cd_split = cd_value.split(';')
            cd.set_values(cd_split)

            cd.set_data_type('list_of_string')

            
        td = network.get_edge_attribute_objects(edge_id, "TISSUE_DATA")
        if td is not None:
            td_value = td.get_values()
            td_split = td_value.split(';')
            td.set_values(td_split)

            td.set_data_type('list_of_string')
            
    template_network = ndex2.create_nice_cx_from_server(server=my_server, 
                                                        uuid=cytoscape_visual_properties_template_id,
                                                        username= my_username, password=my_password)
        
    network.set_network_attribute("labels", template_network.get_network_attribute('labels'))
    network.set_network_attribute("author", template_network.get_network_attribute('author'))
    append_desc = '<p><br/></p><h6><b>Node Legend:</b><br/>Light green oval &gt; Protein/Protein Family<br/>Dark green round rectangle &gt; Complex<br/>Orange octagon &gt; Chemical<br/>Purple octagon &gt; Small molecule<br/>White rectangles &gt; Phenotype<br/>Light blue diamond &gt; Stimulus</h6><h6><b>Edge Legend:</b><br/>Solid &gt; Direct interaction<br/>Dashed &gt; Indirect or Unknown interaction<br/>Blue &gt; Up-regulation<br/>Red &gt; Down-regulation<br/>Black &gt; Form complex or Unknown</h6>'
    network.set_network_attribute('description', 'FULL-Human (' + f"{datetime.now():%d-%b-%Y}" + ') ' + append_desc)
    #network.set_network_attribute("rightsHolder", template_network.get_network_attribute('rightsHolder'))
    #network.set_network_attribute("rights", template_network.get_network_attribute('rights'))
    #network.set_network_attribute("reference", template_network.get_network_attribute('reference'))

    network.set_network_attribute("version", f"{datetime.now():%d-%b-%Y}") #"0.0.1")

    network.set_network_attribute('rightsHolder', 'Prof. Gianni Cesareni')
    network.set_network_attribute("rights", "Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)")
    network.set_network_attribute("reference", "<div>Perfetto L., <i>et al.</i></div><div><b>SIGNOR: a database of causal relationships between biological entities</b><i>.</i></div><div>Nucleic Acids Res. 2016 Jan 4;44(D1):D548-54</div><div><span><a href=\"https://doi.org/10.1093/nar/gkv1048\" target=\"\">doi: 10.1093/nar/gkv1048</a></span></div>")    

    print(network.get_summary())
    return network

process_full_signor(cytoscape_visual_properties_template_id, load_plan, my_server, my_username, my_password)


#signor_network = get_full_signor_network(load_plan)

Name: Untitled
Nodes: 4896
Edges: 19532
Node Attributes: 4896
Edge Attributes: 123621

updating


''